In [2]:
import os
from PIL import Image, ImageEnhance, ImageFilter
import numpy as np

def enhance_blue_noise(input_path, output_path, blue_threshold=150, saturation_factor=1.5, brightness_factor=1.2):
    """深度增强蓝绿色噪点图片，使蓝色更接近纯蓝色(0,0,255)"""
    # 打开图片
    image = Image.open(input_path)
    
    # 1. 提高整体对比度
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(1.8)
    
    # 2. 转换为HSV颜色空间以更好地控制饱和度和亮度
    hsv_image = image.convert('HSV')
    hsv_array = np.array(hsv_image)
    
    # 3. 提取通道
    h, s, v = hsv_array[:,:,0], hsv_array[:,:,1], hsv_array[:,:,2]
    
    # 4. 找到蓝色区域 (色相范围约为180-240度，HSV中色相范围是0-179)
    # 蓝色在HSV中大约是100-120
    blue_mask = (h >= 100) & (h <= 120)
    
    # 5. 对蓝色区域进行处理
    # 提高蓝色区域的饱和度
    s[blue_mask] = np.minimum(s[blue_mask] * saturation_factor, 255).astype(np.uint8)
    
    # 提高蓝色区域的亮度
    v[blue_mask] = np.minimum(v[blue_mask] * brightness_factor, 255).astype(np.uint8)
    
    # 6. 应用阈值处理，使蓝色更接近纯蓝色
    # 将符合蓝色条件的像素直接设置为接近(0,0,255)的颜色
    rgb_array = np.array(image)
    r, g, b = rgb_array[:,:,0], rgb_array[:,:,1], rgb_array[:,:,2]
    
    # 找到符合条件的蓝色像素 (B > G+50 且 B > R+50)
    blue_condition = (b > g + 50) & (b > r + 50) & (b > blue_threshold)
    
    # 将符合条件的像素直接设置为接近(0,0,255)的颜色
    r[blue_condition] = 0
    g[blue_condition] = 0
    b[blue_condition] = 255
    
    # 7. 合并处理后的通道
    enhanced_array = np.stack([r, g, b], axis=2)
    enhanced_image = Image.fromarray(enhanced_array)
    
    # 8. 应用锐化滤镜使噪点更明显
    enhanced_image = enhanced_image.filter(ImageFilter.SHARPEN)
    
    # 保存增强后的图片
    enhanced_image.save(output_path)
    
    print(f"已处理: {input_path} -> {output_path}")

def main():
    # 设置基础路径和输出目录
    base_path = "."  # 当前目录
    output_dir = "enhanced"
    
    # 确保输出目录存在
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 处理noise1.bmp到noise10.bmp
    for i in range(1, 11):
        input_file = f"noise{i}.bmp"
        input_path = os.path.join(base_path, input_file)
        
        # 检查文件是否存在
        if os.path.exists(input_path):
            output_file = f"noise{i}.bmp"
            output_path = os.path.join(output_dir, output_file)
            
            # 深度增强蓝绿色噪点
            enhance_blue_noise(input_path, output_path)
        else:
            print(f"文件不存在: {input_path}")

if __name__ == "__main__":
    main()


已处理: ./noise1.bmp -> enhanced/noise1.bmp
已处理: ./noise2.bmp -> enhanced/noise2.bmp
已处理: ./noise3.bmp -> enhanced/noise3.bmp
已处理: ./noise4.bmp -> enhanced/noise4.bmp
已处理: ./noise5.bmp -> enhanced/noise5.bmp
已处理: ./noise6.bmp -> enhanced/noise6.bmp
已处理: ./noise7.bmp -> enhanced/noise7.bmp
已处理: ./noise8.bmp -> enhanced/noise8.bmp
已处理: ./noise9.bmp -> enhanced/noise9.bmp
已处理: ./noise10.bmp -> enhanced/noise10.bmp
